## Start with Importing the Data.

In [6]:
import pandas as pd
import numpy as np

df = pd.read_excel("/content/AirQualityUCI.xlsx")

## Now sort the time and date features.

In [7]:
df['datetime'] = df['Date'].astype(str) + ' ' + df['Time'].astype(str)
df['datetime'] = pd.to_datetime(df['datetime'])
df = df.drop(['Date', 'Time'], axis=1)
df = df.set_index('datetime')
df = df.sort_index()
print(df.head())

                     CO(GT)  PT08.S1(CO)  NMHC(GT)   C6H6(GT)  PT08.S2(NMHC)  \
datetime                                                                       
2004-03-10 18:00:00     2.6      1360.00       150  11.881723        1045.50   
2004-03-10 19:00:00     2.0      1292.25       112   9.397165         954.75   
2004-03-10 20:00:00     2.2      1402.00        88   8.997817         939.25   
2004-03-10 21:00:00     2.2      1375.50        80   9.228796         948.25   
2004-03-10 22:00:00     1.6      1272.25        51   6.518224         835.50   

                     NOx(GT)  PT08.S3(NOx)  NO2(GT)  PT08.S4(NO2)  \
datetime                                                            
2004-03-10 18:00:00    166.0       1056.25    113.0       1692.00   
2004-03-10 19:00:00    103.0       1173.75     92.0       1558.75   
2004-03-10 20:00:00    131.0       1140.00    114.0       1554.50   
2004-03-10 21:00:00    172.0       1092.00    122.0       1583.75   
2004-03-10 22:00:00    13

## Now we check and clear the missing Data (if any)

In [8]:
df.replace(-200, np.nan, inplace=True)
# print(df.isnull().sum())

df.fillna(method='ffill', inplace=True)
df.fillna(method='bfill', inplace=True)
# print(df.isnull().sum())
# print(df.head())

<ipython-input-8-a1cc8e2dd04c>:4: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)
<ipython-input-8-a1cc8e2dd04c>:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='bfill', inplace=True)


## Add some time based features

I did this because the models for XGBoost and LSTM did not give proper output values and the Root Mean Square error was very high.

In [9]:
df['hour'] = df.index.hour
df['dayofweek'] = df.index.dayofweek
df['month'] = df.index.month
df['is_weekend'] = df.index.weekday >= 5

## Now splitting the Data

In [10]:
target_cols = df.columns.tolist()[:-4]

df[target_cols] = df[target_cols].shift(-1)

df.dropna(inplace=True)



X = df.drop(target_cols, axis=1)
y = df[target_cols]

X_train = X[:-48]
y_train = y[:-48]

X_test = X[-48:]
y_test = y[-48:]

## Trying the XGBoost regression with multivariate values

In [11]:
from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor

xgb_model = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=6, random_state=42)

multi_output_model = MultiOutputRegressor(xgb_model)

multi_output_model.fit(X_train, y_train)

y_pred = multi_output_model.predict(X_test)

y_pred_df = pd.DataFrame(y_pred, columns=target_cols)
y_pred_df.index = pd.date_range(start=df.index[-48] + pd.Timedelta(hours=1), periods=48, freq='H')

<ipython-input-11-2f76106f21d2>:18: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  y_pred_df.index = pd.date_range(start=df.index[-48] + pd.Timedelta(hours=1), periods=48, freq='H')


### Check RSME for XGBoost

In [12]:
from sklearn.metrics import mean_squared_error

for col in target_cols:
    rmse = np.sqrt(mean_squared_error(y_test[col], y_pred_df[col]))
    print("RMSE:", rmse, "for", col)

RMSE: 1.0050584527823456 for CO(GT)
RMSE: 152.66218231371985 for PT08.S1(CO)
RMSE: 92.98868844644052 for NMHC(GT)
RMSE: 4.120369255270166 for C6H6(GT)
RMSE: 179.4525013855002 for PT08.S2(NMHC)
RMSE: 138.19659718202803 for NOx(GT)
RMSE: 280.04896266216343 for PT08.S3(NOx)
RMSE: 46.3370312486241 for NO2(GT)
RMSE: 459.72616110618776 for PT08.S4(NO2)
RMSE: 308.30679434265113 for PT08.S5(O3)
RMSE: 4.648979495186465 for T
RMSE: 16.834170092718992 for RH
RMSE: 0.2793818552444672 for AH


## Now trying the LSTM approach

In [17]:
import numpy as np
import tensorflow as tf
LSTM = tf.keras.layers.LSTM
Sequential = tf.keras.models.Sequential
Dense = tf.keras.layers.Dense
TimeDistributed = tf.keras.layers.TimeDistributed
from sklearn.preprocessing import StandardScaler


scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_test = X[-100:]
y_test = y[-100:]

X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

y_train_scaled = scaler_y.fit_transform(y_train)
y_test_scaled = scaler_y.transform(y_test)

window_size = 48  # last 48 hours
forecast_horizon = 48  # next 48 hours
n_features = X_train.shape[1]
n_targets = y_train.shape[1]


def create_sequences(X, y, window_size, n_steps_out):
    Xs, ys = [], []
    for i in range(len(X) - window_size - n_steps_out + 1):
        Xs.append(X[i:i + window_size])
        ys.append(y[i + window_size:i + window_size + n_steps_out])
    return np.array(Xs), np.array(ys)

X_seq_train, y_seq_train = create_sequences(X_train_scaled, y_train_scaled, window_size, forecast_horizon)
X_seq_test, y_seq_test = create_sequences(X_test_scaled, y_test_scaled, window_size, forecast_horizon)

print(y_seq_test.shape)

model = Sequential()
model.add(LSTM(64, activation='relu', return_sequences=True, input_shape=(window_size, n_features)))
model.add(TimeDistributed(Dense(n_targets)))  # Output: (samples, 48, 13)
model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.summary()

history = model.fit(X_seq_train, y_seq_train, epochs=20, batch_size=32, validation_split=0.1)

def predict_full_sequence(model, X_input):
    y_pred = model.predict(X_input)

    return y_pred

X_input = X_seq_test[0:1]
y_true = y_seq_test[0]

y_pred = predict_full_sequence(model, X_input)

y_pred_rescaled = scaler_y.inverse_transform(y_pred[0])
y_true_rescaled = scaler_y.inverse_transform(y_true)

(5, 48, 13)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_4 (LSTM)                   │ (None, 48, 64)         │        17,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_4              │ (None, 48, 13)         │           845 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 18,509 (72.30 KB)

 Trainable params: 18,509 (72.30 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
260/260 ━━━━━━━━━━━━━━━━━━━━ 14s 35ms/step - loss: 0.8631 - mae: 0.6926 - val_loss: 0.6655 - val_mae: 0.6312
Epoch 2/20
260/260 ━━━━━━━━━━━━━━━━━━━━ 9s 35ms/step - loss: 0.6180 - mae: 0.5658 - val_loss: 0.6518 - val_mae: 0.6258
Epoch 3/20
260/260 ━━━━━━━━━━━━━━━━━━━━ 10s 36ms/step - loss: 0.5675 - mae: 0.5386 - val_loss: 0.7431 - val_mae: 0.6660
Epoch 4/20
260/260 ━━━━━━━━━━━━━━━━━━━━ 9s 36ms/step - loss: 0.5449 - mae: 0.5263 - val_loss: 0.7056 - val_mae: 0.6472
Epoch 5/20
260/260 ━━━━━━━━━━━━━━━━━━━━ 8s 31ms/step - loss: 0.5283 - mae: 0.5166 - val_loss: 0.7241 - val_mae: 0.6514
Epoch 6/20
260/260 ━━━━━━━━━━━━━━━━━━━━ 11s 34ms/step - loss: 0.5182 - mae: 0.5116 - val_loss: 0.6778 - val_mae: 0.6334
Epoch 7/20
260/260 ━━━━━━━━━━━━━━━━━━━━ 11s 36ms/step - loss: 0.5061 - mae: 0.5041 - val_loss: 0.7301 - val_mae: 0.6490
Epoch 8/20
260/260 ━━━━━━━━━━━━━━━━━━━━ 10s 37ms/step - loss: 0.5003 - mae: 0.5011 - val_loss: 0.7466 - val_mae: 0.6538
Epoch 9/20
260/260 ━━━━━━━━━━━━━━━━━━━━ 9s 

In [20]:
rmse = np.sqrt(mean_squared_error(y_true_rescaled, y_pred_rescaled))
print(f"RMSE for the full 48-hour forecast: {rmse:.2f}")
print('---------------------------------------')

rmse_each = np.sqrt(mean_squared_error(y_true_rescaled, y_pred_rescaled, multioutput='raw_values'))
for i, col in enumerate(target_cols):
    print(f"{col} RMSE for 48-hour forecast: {rmse_each[i]:.2f}")

RMSE for the full 48-hour forecast: 215.29
---------------------------------------
CO(GT) RMSE for 48-hour forecast: 1.08
PT08.S1(CO) RMSE for 48-hour forecast: 207.97
NMHC(GT) RMSE for 48-hour forecast: 55.39
C6H6(GT) RMSE for 48-hour forecast: 4.88
PT08.S2(NMHC) RMSE for 48-hour forecast: 209.91
NOx(GT) RMSE for 48-hour forecast: 141.37
PT08.S3(NOx) RMSE for 48-hour forecast: 267.73
NO2(GT) RMSE for 48-hour forecast: 43.12
PT08.S4(NO2) RMSE for 48-hour forecast: 505.83
PT08.S5(O3) RMSE for 48-hour forecast: 403.13
T RMSE for 48-hour forecast: 3.42
RH RMSE for 48-hour forecast: 14.84
AH RMSE for 48-hour forecast: 0.25


Final Addition to submissions.xlsx

In [22]:
y_pred_df['Date'] = y_pred_df.index.date
y_pred_df['Time'] = y_pred_df.index.time

cols = ['Date', 'Time'] + [col for col in y_pred_df.columns if col not in ['Date', 'Time']]
y_pred_df = y_pred_df[cols]


final = y_pred_df.reset_index(drop=True)
print(final.head())

final.to_excel('/content/submission.xlsx', index=False)

         Date      Time    CO(GT)  PT08.S1(CO)    NMHC(GT)   C6H6(GT)  \
0  2005-04-02  15:00:00  2.017102  1118.574341  232.940613   8.187978   
1  2005-04-02  16:00:00  2.021621  1157.741821  235.140320   8.639378   
2  2005-04-02  17:00:00  2.167089  1173.840576  241.334518   9.639607   
3  2005-04-02  18:00:00  2.489129  1251.551758  253.496033  11.061991   
4  2005-04-02  19:00:00  2.897300  1332.381958  307.464020  13.679197   

   PT08.S2(NMHC)     NOx(GT)  PT08.S3(NOx)     NO2(GT)  PT08.S4(NO2)  \
0     880.815674  111.343872    982.837463   89.071510   1456.787476   
1     910.425049  117.146156    958.458435   93.807205   1484.963623   
2     939.865540  127.196983    912.675232   94.937973   1543.387573   
3    1005.164490  138.627121    878.738464  100.714485   1652.246582   
4    1096.087769  149.299347    784.343811  104.428116   1723.625732   

   PT08.S5(O3)          T         RH        AH  
0   716.247925  21.994081  34.559536  0.879285  
1   755.287048  22.022842  34.

In [23]:
import pandas as pd
from datetime import timedelta

y_pred_df = pd.DataFrame(y_pred_rescaled, columns=target_cols)

last_datetime = df.index[-1]  # Assumes df has a DateTimeIndex
forecast_datetimes = [last_datetime + timedelta(hours=i+1) for i in range(48)]
y_pred_df.index = pd.to_datetime(forecast_datetimes)

y_pred_df['Date'] = y_pred_df.index.date
y_pred_df['Time'] = y_pred_df.index.time

cols = ['Date', 'Time'] + [col for col in y_pred_df.columns if col not in ['Date', 'Time']]
y_pred_df = y_pred_df[cols]

final = y_pred_df.reset_index(drop=True)
print(final.head())

final.to_excel('submission_lstm_48hr.xlsx', index=False)


         Date      Time    CO(GT)  PT08.S1(CO)    NMHC(GT)   C6H6(GT)  \
0  2005-04-04  14:00:00  1.911380  1103.150513  270.984589   8.945854   
1  2005-04-04  15:00:00  2.057703  1137.935303  274.841187   9.137772   
2  2005-04-04  16:00:00  2.279827  1181.178101  284.902252   9.852685   
3  2005-04-04  17:00:00  2.421160  1210.083740  297.946777  10.338468   
4  2005-04-04  18:00:00  2.483078  1224.153076  310.863556  10.552594   

   PT08.S2(NMHC)     NOx(GT)  PT08.S3(NOx)     NO2(GT)  PT08.S4(NO2)  \
0     907.589844  202.073792    856.781250  114.025490   1436.043945   
1     907.235962  213.983459    861.168457  124.845520   1409.673218   
2     930.334167  225.548386    871.587219  130.304733   1418.733643   
3     948.341553  218.447693    889.779663  129.164734   1444.927612   
4     957.266479  199.025970    905.348755  124.877251   1471.379272   

   PT08.S5(O3)          T         RH        AH  
0   976.768494  17.991461  46.094986  0.929513  
1   979.436829  16.449894  45.